<a href="https://colab.research.google.com/github/Nikhil-4-Pal/CareNexus/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.__version__)


2.6.0+cu124


In [ ]:
if torch.cuda.is_available():
  print("GPU is avialable")
  print(f"using gpu : {torch.cuda.get_device_name(0)}")

else :
  print("GPU is not available")

GPU is avialable
using gpu : Tesla T4


# Creating Tensor

In [ ]:
# Using empty
a = torch.empty(2,3)

In [ ]:
# check type

type(a)

torch.Tensor

In [ ]:
# using zeros

torch.zeros(2,3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [ ]:
# using ones
torch.ones(2,3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [ ]:
# using rand
torch.rand(2,3)

tensor([[0.5234, 0.1381, 0.4138],
        [0.7802, 0.7608, 0.2380]])

In [ ]:
# using seed to reproduce same random tensor

torch.manual_seed(100)
torch.rand(2,3)

tensor([[0.1117, 0.8158, 0.2626],
        [0.4839, 0.6765, 0.7539]])

In [ ]:
# using tensor

torch.tensor([[1,2,3],[2,3,4]])

tensor([[1, 2, 3],
        [2, 3, 4]])

# Tensor Shapes

In [ ]:
x = torch.tensor([[[1,2],[3,4]],[[4,5],[6,7]]])
x.shape

torch.Size([2, 2, 2])

In [ ]:
torch.empty_like(x)

tensor([[[132552189087120,       153957872],
         [              0,               0]],

        [[              0,               0],
         [              0,               0]]])

In [ ]:
torch.zeros_like(x)

tensor([[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]])

 # Tensor Data Type

In [ ]:
x.dtype

torch.int64

In [ ]:
# assign the dtype
torch.tensor([12,3,4,5,6],dtype=torch.float32)

tensor([12.,  3.,  4.,  5.,  6.])

In [ ]:
# change the dtype

x.to(torch.float32)

tensor([[[1., 2.],
         [3., 4.]],

        [[4., 5.],
         [6., 7.]]])

In [ ]:
torch.rand_like(x,dtype=torch.float32)

tensor([[[0.2627, 0.0428],
         [0.2080, 0.1180]],

        [[0.1217, 0.7356],
         [0.7118, 0.7876]]])

# reshaping tensors

In [ ]:
# reshape
x.reshape(4,2)

tensor([[1, 2],
        [3, 4],
        [4, 5],
        [6, 7]])

In [ ]:
# flatten
x.flatten()

tensor([1, 2, 3, 4, 4, 5, 6, 7])

# AutoGrad

In [ ]:
x = torch.tensor(3.0 , requires_grad=True)

In [ ]:
y = x**2

In [ ]:
x , y

(tensor(3., requires_grad=True), tensor(9., grad_fn=<PowBackward0>))

In [ ]:
y.backward()

In [ ]:
x.grad

tensor(6.)

In [ ]:
y = x**2

In [ ]:
z = torch.sin(y)


In [ ]:
y , z

(tensor(9., grad_fn=<PowBackward0>), tensor(0.4121, grad_fn=<SinBackward0>))

In [ ]:
z.backward()

In [ ]:
x.grad

tensor(0.5332)

In [ ]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0)
b = torch.tensor(0.0)

In [ ]:
# Binary Cross-Entropy loss for scalar

def binary_cross_entropy_loss( prediction , target):
  epsilon = 1e-8
  prediction = torch.clamp(prediction , epsilon , 1-epsilon)
  return -(target * torch.log(prediction)+(1-target)*torch.log(1-prediction))

In [ ]:
# forward pass

z = w*x + b
y_pred = torch.sigmoid(z)

# compute the loss

loss = binary_cross_entropy_loss(y_pred , y)

In [ ]:
# Derivatives

dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

dy_pred_dz = y_pred*(1-y_pred)

dz_dw = x
dz_db = 1

dl_dw  = dloss_dy_pred * dy_pred_dz * dz_dw
dl_db = dloss_dy_pred * dy_pred_dz * dz_db

In [ ]:
print(f"mamual gradient of loss wrt weight : {dl_dw}")
print(f"mamual gradient of loss wrt bias : {dl_db}")

mamual gradient of loss wrt weight : 6.691762447357178
mamual gradient of loss wrt bias : 0.998770534992218


# Training Pipeline

## flow of the code

1. Load the Dataset
2. Basic Programming
3. Training Process

    *  Create the model
    * Forward Pass
    * Loss Calculation
    * Backpropagation
    * Parameters Update
4. Model Evaluation





In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

df.head()



,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [ ]:
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
# train_test_split

x_train , x_test , y_train , y_test = train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size = 0.2)

In [ ]:
x_train.shape ,y_train.shape,x_test.shape,y_test.shape

((455, 30), (455,), (114, 30), (114,))

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train

array([[ 0.90383336,  0.66478783,  0.93412053, ...,  0.5797624 ,
         0.28418285, -0.50066713],
       [-0.23248913,  0.23767902, -0.18249409, ...,  0.52445309,
         1.43125348,  1.02368797],
       [-0.61220787, -0.23329503, -0.66121993, ..., -0.80051203,
        -0.19717715, -0.5422504 ],
       ...,
       [-0.77656374, -0.38797768, -0.79399316, ..., -0.43147606,
         0.12372951, -0.35786143],
       [-0.80206724, -1.45228668, -0.76636643, ...,  0.25712479,
         0.75530114,  0.44644654],
       [ 1.64060105,  0.23306162,  1.59798668, ...,  0.71035381,
        -0.60001743,  0.40322182]])

In [ ]:
# label encoding

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

array([1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,

In [ ]:
# numpy arrays to pytorch tenosor

x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [ ]:
x_train_tensor.shape

torch.Size([455, 30])

 # Defining The model

In [ ]:
class MySimpleNN():

  def __init__(self, x):
    self.weights = torch.rand(x.shape[1],1 , dtype=torch.float64 , requires_grad=True)
    self.bias = torch.zeros(1 , dtype=torch.float64 , requires_grad=True)

  def forward(self , x):
    z = torch.matmul(x  , self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self , y_pred , y):
    # clamp prediction to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred , epsilon , 1-epsilon)

    # calculate loss
    loss = -(y_train_tensor*torch.log(y_pred) + (1-y_train_tensor)* torch.log(1-y_pred) ).mean()
    return loss

# important parameters

In [ ]:
learning_rate = 0.1
epochs = 25

# Training Pipeline

In [ ]:
# create model

model = MySimpleNN(x_train_tensor)

# define loop
for epoch in range(epochs):
  # forward pass
  y_pred = model.forward(x_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred , y_train_tensor)

  # backward pass
  loss.backward()
  # parameters update
  with torch.no_grad():
    model.weights -= learning_rate*weights.grad
    model.bias -= learning_rate*bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss im each epoch
  print(f'epoch:{epoch +1},loss : {loss.item()}')

AttributeError: 'MySimpleNN' object has no attribute 'weight'

 # PyTorch NN Module

In [ ]:
 # create a model class
 import torch
 import torch.nn as nn

 class Model(nn.Module):

  def __init__(self, num_features):

    super().__init__()

    self.Linear = nn.Linear(num_features , 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self , features):

    out = self.sigmoid(features)
    out = self.sigmoid(out)

    return out

In [ ]:
# create dataset

features = torch.rand(10,5)

# create model

model = Model(features.shape[1])

# call model for forward pass
model(features)

tensor([[0.6742, 0.6713, 0.6371, 0.6465, 0.6705],
        [0.6460, 0.6440, 0.6346, 0.6694, 0.6246],
        [0.6330, 0.6580, 0.6615, 0.6476, 0.6625],
        [0.6706, 0.6462, 0.6495, 0.6350, 0.6397],
        [0.6271, 0.6713, 0.6587, 0.6471, 0.6581],
        [0.6633, 0.6728, 0.6691, 0.6237, 0.6318],
        [0.6534, 0.6577, 0.6590, 0.6542, 0.6623],
        [0.6676, 0.6452, 0.6643, 0.6572, 0.6441],
        [0.6667, 0.6727, 0.6748, 0.6573, 0.6571],
        [0.6556, 0.6275, 0.6549, 0.6400, 0.6715]])

In [ ]:
 model.Linear.bias

Parameter containing:
tensor([0.1533], requires_grad=True)

In [ ]:
 model.Linear.weight

Parameter containing:
tensor([[ 0.3259, -0.3946,  0.1052, -0.3751, -0.0527]], requires_grad=True)

# improved model

In [ ]:
class Model(nn.Module):

  def __init__(self, num_features):

    super().__init__()

    self.linear1 = nn.Linear(num_features,3)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(3,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self , features):
    out = self.linear1(features)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)

    return out

In [ ]:
features = torch.rand(10,5)

model = Model(features.shape[1])

model(features)

tensor([[0.3330],
        [0.3946],
        [0.3545],
        [0.3732],
        [0.4045],
        [0.3755],
        [0.3497],
        [0.3877],
        [0.3778],
        [0.3267]], grad_fn=<SigmoidBackward0>)

In [ ]:
model.linear1.weight  ,model.linear2.weight ,model.linear1.bias  ,model.linear2.bias

(Parameter containing:
 tensor([[-0.3297,  0.2443, -0.0492,  0.3370, -0.1509],
         [ 0.0256,  0.0746, -0.2276,  0.1594,  0.0878],
         [ 0.1888,  0.4227, -0.3566,  0.1190,  0.0413]], requires_grad=True),
 Parameter containing:
 tensor([[-0.0461, -0.3993, -0.5103]], requires_grad=True),
 Parameter containing:
 tensor([-0.3309,  0.3504,  0.1105], requires_grad=True),
 Parameter containing:
 tensor([-0.2196], requires_grad=True))

# Dataloader

In [11]:
from sklearn.datasets import make_classification
import torch
from torch.utils.data import Dataset , DataLoader

In [9]:
# Step 1 : create a synthetic classification dataset using sklearn

x , y = make_classification(
    n_samples=10,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_classes=2,
    random_state=42

)

In [8]:
x , y

(array([[-0.79177382,  1.00384532],
        [-2.06630767,  0.23252327],
        [ 0.63526322, -0.533996  ],
        [ 0.87988045,  0.73763983],
        [ 1.09874229,  1.26367946],
        [ 1.28120131, -1.17877011],
        [-0.36427601,  1.89012707],
        [ 0.59994324, -0.48645009],
        [-1.09975518, -0.96980479],
        [ 0.26126527, -0.06078509]]),
 array([1, 0, 0, 1, 1, 1, 0, 0, 0, 1]))

In [10]:
x = torch.tensor(x , dtype = torch.float32)
y = torch.tensor(y , dtype = torch.float32)

In [12]:
class CustomDataset(Dataset):

  def __init__(self, features , labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return self.features.shape[0]

  def __getitem__(self , index):

    return self.features[index],self.labels[index]


In [13]:
dataset = CustomDataset(x,y)

In [14]:
len(dataset)

10

In [16]:
dataloader = DataLoader(dataset , batch_size=2,shuffle=True)

In [18]:
for batch_features , batch_labels in dataloader :
  print(batch_features)
  print(batch_labels)
  print("-"*50)

tensor([[ 1.7774,  1.5116],
        [-2.8954,  1.9769]])
tensor([1., 0.])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388]])
tensor([1., 0.])
--------------------------------------------------
tensor([[-0.9382, -0.5430],
        [-0.5872, -1.9717]])
tensor([1., 0.])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [ 1.8997,  0.8344]])
tensor([0., 1.])
--------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [-1.9629, -0.9923]])
tensor([1., 0.])
--------------------------------------------------


# Building and ANN in pytorch


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset , DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt

In [2]:
# set a random seed for reproducibility

torch.manual_seed(42)

In [26]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))]
)

In [27]:
train_dataset = FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

In [28]:
test_dataset = FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform = transform
)

In [29]:
train_loader = DataLoader(dataset=train_dataset , batch_size=32 , shuffle = True)
test_loader = DataLoader(dataset=test_dataset , batch_size=32 , shuffle = False)

In [37]:
train_img , train_label = next(iter(train_loader))
test_img , test_label = next(iter(test_loader))

In [54]:
# define NN class

class MyNN(nn.Module):

  def __init__(self , num_features):

    super().__init__()
    self.model  = nn.Sequential(
        nn.Flatten(),
        nn.Linear(num_features,128),
        nn.LeakyReLU(),
        nn.Linear(128,64),
        nn.LeakyReLU(),
        nn.Linear(64,10)
    )


  def forward(self , x):
    return self.model(x)

In [55]:
# set learning rate and epochs
learning_rate = 0.1
epochs = 100

In [58]:
# instantiate the model

model = MyNN(train_img.shape[1]*train_img.shape[2]*train_img.shape[3])

# loss funciton

criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr = learning_rate)


In [60]:
# training loop
total_epoch_loss = 0
for epoch in range(epochs):
  for batch_features , batch_labels in train_loader :

    # forward pass
    output = model(batch_features)
    # calculate loss
    loss = criterion(output,batch_labels)
    # back pass
    optimizer.zero_grad()
    loss.backward()
    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f"epoch: {epoch +1} , loss :{avg_loss}")

epoch: 1 , loss :0.5421899160385132
epoch: 2 , loss :0.9331291411995888
epoch: 3 , loss :1.2842741196235021
epoch: 4 , loss :1.6113073880374431
epoch: 5 , loss :1.9201655858755111
epoch: 6 , loss :2.212873085097472
epoch: 7 , loss :2.4915063673615454
epoch: 8 , loss :2.7603063930859166
epoch: 9 , loss :3.0214785086294014
epoch: 10 , loss :3.2704557450989884
epoch: 11 , loss :3.511386270653208
epoch: 12 , loss :3.744261793149511
epoch: 13 , loss :3.9712227050935227
epoch: 14 , loss :4.191541303832829
epoch: 15 , loss :4.406590209219605
epoch: 16 , loss :4.61593244083797
epoch: 17 , loss :4.8189005048103635
epoch: 18 , loss :5.013812221667916
epoch: 19 , loss :5.205508744064718
epoch: 20 , loss :5.391166447532922
epoch: 21 , loss :5.573815043081591
epoch: 22 , loss :5.7504478413182
epoch: 23 , loss :5.924284386356423
epoch: 24 , loss :6.090755681167046
epoch: 25 , loss :6.254959286144376
epoch: 26 , loss :6.415918333878617
epoch: 27 , loss :6.575217651176701
epoch: 28 , loss :6.729374728